In [ ]:
import os
import sys
import numpy as np
import warnings
from astropy.io import fits
from astropy.time import Time
import math
import glob
import matplotlib.pyplot as plt
%matplotlib inline
prefolder='../../examples/V1/recipe_results/'

In [ ]:
MJD_TO_JD = 2400000.5
class RadialVelocityStats:
    """ This module defines class ' RadialVelocityStats' and methods to do statistic analysis on radial velocity
    results. (this is currently for radial velocity development and testing only).

    Attributes:
         rv_result_set (list): A container storing radial velocity result from fits of level 1 data.
         total_set (int): Total elements in `rv_result_set`.
    """

    def __init__(self, obs_rv_results: list = None):
        self.rv_result_set = list() if obs_rv_results is None else obs_rv_results.copy()
        self.total_set = 0 if obs_rv_results is None else len(obs_rv_results)

    def get_collection(self):
        return self.rv_result_set, self.total_set

    def add_data(self, ccf_rv: float, obj_jd: float):
        self.rv_result_set.append({'jd': obj_jd, 'mean_rv': ccf_rv})
        self.total_set = len(self.rv_result_set)
        return self.rv_result_set, self.total_set

    def analyze_multiple_ccfs(self, ref_date=None):
        """ Statistic analysis on radial velocity numbers of multiple observation resulted by `RadialVelocityAlg`.

        Args:
            ref_date (str, optional): Reference time in the form Julian date format.  Defaults to None.

        Returns:
            dict: Analysis data.

        """
        obs_rvs, total_obs = self.get_collection()
        jd_list = np.array([obs_rv['jd'] for obs_rv in obs_rvs])
        if ref_date is None:
            ref_jd = self.get_start_day(jd_list)
        else:
            ref_jd = Time(ref_date, format='isot', scale='utc').jd
        rv_stats = dict()
        rv_stats['start_jd'] = ref_jd
        rv_stats['hour'] = (jd_list-ref_jd) * 24.0
        rv_stats['day'] = (jd_list-ref_jd)
        rv_stats['values'] = np.array([obs_rv['mean_rv'] for obs_rv in obs_rvs])
        rv_stats['mean'] = np.mean(rv_stats['values'])
        rv_stats['sigma'] = np.std(rv_stats['values'] - rv_stats['mean'])

        return rv_stats

    @staticmethod
    def get_start_day(jd_list: np.ndarray):
        min_jd = np.amin(jd_list)
        day_part = math.floor(min_jd - MJD_TO_JD)
        return MJD_TO_JD+day_part


In [ ]:
def f_decimal(num):
    return "{:.16f}".format(num)

In [ ]:
def plot_velocity_time(rv_info, title, label, color, time_unit='hrs', savefig=None):
    k1 = list(rv_info.keys())[0]
    
    total_rv = np.size(rv_info[k1]['values'])

    if time_unit == 'hrs':
        rv_delta_time = rv_info[k1]['hour'] 
    else:
        rv_delta_time = rv_info[k1]['day']
        
    plt.figure(figsize=(8,6))     
    s = 100
    ymax = -10000
    ymin = 10000
    for k in rv_info.keys():
        rv_offset = (rv_info[k]['values'] - rv_info[k]['mean']) * 1000.0
        ymax = max(np.amax(rv_offset), ymax)
        ymin = min(np.amin(rv_offset), ymin)
        rv_sigma = rv_info[k]['sigma']*1000.0
        plt.scatter(rv_delta_time, rv_offset, s, c=color[k], edgecolors='b', 
                    label=label[k] + r' $\sigma = $'+ "{:0.16f}".format(rv_sigma)+' m/s')
        

    plt.legend(loc="upper right", prop={'size':12})
    plt.xlabel('Times ['+time_unit+']')
    plt.ylabel('RV [m/s]')

    ymax = math.ceil(ymax)+1
    ymin = math.floor(ymin)-1
    xmin = math.ceil(np.amin(rv_delta_time))
    xmax = math.floor(np.amax(rv_delta_time))
    if time_unit == 'hrs':
        plt.xlim((xmin-1, xmax+1))
    else:
        plt.xlim((xmin-20, xmax+20))
    if title is not None:
        plt.title(title)
    
    if savefig is not None:
        plt.savefig(savefig)
    plt.show()    
    

## RV plot from the bootcamp recipes

In [ ]:
"""
Recipes: 
events     | Order trace | Extraction                           | WaveCal | Mask cross-correlation | RV
settings   | Y/          | rect_method: vertical/normal/norect  | Y/N     |                        | weighted/
           |<recipe x>   | extract_method: sum/optimal          |         |                        | unweighted 

RecipeA    | Y           | norect,sum                           | N       | Y                      | unweighted
RecipeB    | RecipeA     | rectified (vertical), sum            | N       | Y                      | weighted
RecipeC    | RecipeA     | norect, sum                          | Y       | Y                      | unweighted
RecipeD    | RecipeA     | rectified (vertical), sum            | Y       | Y                      | unweighted
RecipeE    | RecipeA     | rectified (vertical), sum            | Y       | Y                      | weighted
RecipeF    | Y           | rectified (vertical), optimal        | Y       | Y                      | weighted

Alpha 1    | Y           | norect,optimal                       | N       | Y                      | unweighted
Alpha 2    | from Alpha 1| rectified (vertical), optimal        | N       | Y                      | weighted
"""

In [ ]:
recipe_lev2 = {'RecipeA':'norect_optimal_all_L1_L2', \
               'RecipeB': 'vertical_sum_all_L1_L2', \
               'RecipeC': 'norect_sum_all_L1_wave_L2', \
               'RecipeD': 'vertical_sum_all_L1_wave_L2', \
               'RecipeE': 'vertical_sum_all_L1_wave_L2_reweighted', \
               'RecipeF': 'vertical_optimal_all_L1_wave_L2_reweighted', \
               'RecipeAlpha1':'norect_optimal_all_L1_L2', \
               'RecipeAlpha2':'vertical_optimal_all_L1_L2_reweighted'}
recipe_list = ['', 'RecipeA', 'RecipeB', 'RecipeC', 'RecipeD', 'RecipeE', 'RecipeF', \
                'RecipeAlpha1', 'RecipeAlpha2']

In [ ]:
while True:
    a = input("1-6 for Recipe A- F, 7: alpha 1 test, 8: alpha 2 test, 0: end ")
    a = int(a)
    
    if a == 0:
        break
    if a < 1 or a > 8:
        continue
                
    ridx = recipe_list[int(a)]   # specify the recipe index from the keys of recipe_lev2
    lev2_file_pattern = prefolder + ridx + '/*' + recipe_lev2[ridx] + '.fits'

    rv_neid_info = RadialVelocityStats()
    ccf_idx = 3 if 'reweighted' in recipe_lev2[ridx] else 9

    lev2_files = sorted(glob.glob(lev2_file_pattern))
    
    for f in range(len(lev2_files)):
        print(lev2_files[f])    
        hdulist = fits.open(lev2_files[f])
        rv_val = float(hdulist[ccf_idx].header['CCF-RVC'])
        L1_time = float(hdulist[ccf_idx].header['CCFJDSUM'])
        rv_neid_info.add_data(rv_val, L1_time)
        
    rv_neid_stats = rv_neid_info.analyze_multiple_ccfs()
    rv_stats = {'neid': rv_neid_stats}
    print("rv: ", rv_neid_stats['values'], " mean: ", rv_neid_stats["mean"])
    plot_velocity_time(rv_stats, "rv stats from "+ridx, \
                       {'neid': ""}, \
                       {'neid': 'cyan'}, time_unit='hrs')
        

## RV result test from NEID recipes

In [ ]:
lev2_file_pattern = '/Users/cwang/documents/KPF/KPF-Pipeline/test_results/neid_hd73344/output_rv_v2/*.fits'
#'/Users/cwang/documents/KPF/KPF-Pipeline/test_results/neid_hd73344/output_rv/*.fits'
#'/Users/cwang/documents/KPF/ownCloud/KPF-Pipeline-TestData/DRP_V2_Testing/NEID_HD73344/output_rv/*.fits'
# lev2_file_pattern = "/Users/cwang/documents/KPF/KPF-Pipeline/examples/V1/recipe_results/RecipeA_0903/tmp_0907/*L2_0903.fits"
lev2_files = sorted(glob.glob(lev2_file_pattern))
print(lev2_files)
rv_idx = 5
rv_neid_info = RadialVelocityStats()

for f in range(len(lev2_files)):
    print(lev2_files[f])    
    if 'reweighted' in lev2_files[f]:
        continue
    hdulist = fits.open(lev2_files[f])
    rv = hdulist[rv_idx].header['CCD1RV1']
    idx = str(rv).find('Bary')
    if idx >= 0:
        rv = rv[0:idx]
        
    rv_val = float(rv)
    L1_time = float(hdulist[rv_idx].header['CCD1JD'])
    rv_neid_info.add_data(rv_val, L1_time)
    
rv_neid_stats = rv_neid_info.analyze_multiple_ccfs()
    
rv_stats = {'neid': rv_neid_stats}
rv_v = np.array([f_decimal(v)  for v in rv_neid_stats['values']])
print("rv: ", rv_v, " mean: ", f_decimal(rv_neid_stats["mean"]))
plot_velocity_time(rv_stats, "rv stats from clipping file used in optimal extraction", \
                    {'neid': ""}, \
                    {'neid': 'cyan'}, time_unit='hrs')


## Compare the data from wavelength calibration with the data from NEID 

In [ ]:
level1_wave_files = prefolder+'RecipeC/*_L1_wave.fits'
level1_neid_files = prefolder+'RecipeC/*_L1.fits'

sci_wave_idx = 6
wave_files = sorted(glob.glob(level1_wave_files))
neid_files = sorted(glob.glob(level1_neid_files))
import pdb;pdb.set_trace()
for f in range(len(wave_files)):
    print(wave_files[f])
    print(neid_files[f])

    wave_hdulist = fits.open(wave_files[f])
    neid_hdulist = fits.open(neid_files[f])
    wave_vals =wave_hdulist[sci_wave_idx].data
    neid_vals =neid_hdulist[sci_wave_idx].data
    h, w = np.shape(wave_vals)
    order_set = np.arange(40,60, dtype=int)
    for order in order_set:
        diff = wave_vals[order] - neid_vals[order]
        x_data = np.arange(0, w, dtype=int)
        y_data = diff[x_data]
        plt.figure(figsize=(18,6))
        plt.plot(x_data, y_data, c = 'blue')
        plt.title( 'order '+str(order))
        plt.show()
        
        